## Hodgkin Huxley Simulation Notebook

This Jupyter (python) notebook contains a simple simulation of a single compartment neuron using voltage gated sodium and potassium channels and the equations developed by Hodgkin and Huxley. The parameters are taken from _Theoretical and Computational Neuronscience_ by Dayan and Abbott. You can clone and modify the code to perform the analyses for [Assignment 2 for Rice ELEC-548](https://elec548.github.io/Assignments/hw2.html).

In [1]:
# Begin by importing useful files
import numpy as np
import matplotlib.pyplot as plt

# Matplotlib parameter commands must be run before any matplotlib commands are, 
#   and it is advisable to reload the notebook if you want to change them.
#   User "notebook" for pan and zoom, or "inline" for best rendering.
#%matplotlib notebook 
%matplotlib inline

import seaborn as sns # seaborn styles things nicely
sns.set(rc={'figure.figsize': (12, 4),'lines.linewidth': 2, 'font.size': 18, 
            'axes.labelsize': 16, 'legend.fontsize': 12, 'ytick.labelsize': 12, 
            'xtick.labelsize': 12 })
sns.set_style('white')

In [ ]:
def hh(I, dt):
    ##########################################################################
    # Simulate the membrane potential of a Hodgkin Huxley neuron with 
    #   a given input current
    # Input:
    #    I = current in uA/mm^2a
    #    dt = time step between I measurments [ms]
    #
    # Output:
    #    Vm = membrane voltage in mV
    #    n = sodium activation
    #    m = potassium activation
    #    h = 1 - potassium inactivation
    
    # This function simulates a dynamical system with state variables
    #    DV = 1/C (I - Ik - Ina - Il)
    #    Il = gl(V-El)
    #    Ik = gk*n^4(V-Ek)
    #    Ina = gna*m^3*h(V-Ena)
    #    Dn = (ninf(V) - n)/taun(V) (same for m, h )

    # Summary of units: I = uA / mm^2; V = mV; g = mS; g*V = uA; C = uF; uA/uC*ms = mV

    ##########################################################################
    # Constants:
    # Reversal potentials for various ions
    Ek = -77 #[mV]
    Ena = 50 #[mV]
    El = -54.402 #[mV]

    # Membrane capacitance: 
    C = 0.01 #[uF/mm^2]
    
    # Maximum conductances [mS/mm^2]
    gna = 1.2
    gk = 0.36
    gl = 0.003

    ##########################################################################
    # Gating variables:
    # activation K [n]
    alpha_n = lambda V: 0.01*(V + 55) / (1 - np.exp(-0.1*(V + 55)))
    beta_n = lambda V: 0.125 * np.exp(-0.0125*(V + 65))
    tau_n = lambda V: 1 /(alpha_n(V) + beta_n(V))
    n_inf = lambda V: alpha_n(V) * tau_n(V)
    # activation Na [m]
    alpha_m = lambda V: 0.1*(V + 40) / (1 - np.exp(-0.1*(V + 40)))
    beta_m = lambda V: 4 * np.exp(-0.0556*(V + 65))
    tau_m = lambda V: 1/(alpha_m(V) + beta_m(V))
    m_inf = lambda V: alpha_m(V) * tau_m(V)
    # inactivation Na [h]
    alpha_h = lambda V: 0.07*np.exp(-0.05*(V + 65))
    beta_h = lambda V:  1 /(1 + np.exp(-0.1*(V + 35)))
    tau_h = lambda V: 1/(alpha_h(V) + beta_h(V))
    h_inf = lambda V: alpha_h(V) * tau_h(V);
    
    # Initializations
    n = np.zeros(len(I)); m = np.zeros(len(I)); h = np.zeros(len(I)); V = np.zeros(len(I))
 
    # Set initial conditions:
    Vstart = -65 #[mV] (starting membrane potential) 
    V[0] = Vstart #[mV]
    n[0] = n_inf(Vstart); m[0] = m_inf(Vstart); h[0] = h_inf(Vstart);
    
    ##########################################################################
    # Simulation: iteratatively update the variables using the forward Euler method
    for ii in range(len(I)-1):
        # Update activation state variables
        n[ii+1] = n[ii] + dt*(n_inf(V[ii]) - n[ii])/tau_n(V[ii])
        m[ii+1] = m[ii] + dt*(m_inf(V[ii]) - m[ii])/tau_m(V[ii])
        h[ii+1] = h[ii] + dt*(h_inf(V[ii]) - h[ii])/tau_h(V[ii])
        V[ii+1] = V[ii] + dt/C*(I[ii] - gl*(V[ii]-El) - gk*n[ii]**4 *(V[ii]-Ek) - gna*m[ii]**3*h[ii]*(V[ii]-Ena));

    return V, m, n, h


The code above implements the Hodgkin Huxley model using the Forward Euler method to solve the differential equations. For your assignment, you will test the effects of various parameters on the solution, and modify it by adding an addition voltage-gated ion channel. In the following cells, we inject a current pulse to demonstrate that this depolarization causes and action potential in the model.

In [ ]:
dt = 1e-3; # 1 us
I = np.zeros(int(1000 / dt)) # 1 s of zero current
I[int(200/dt):int(210/dt)] = 0.2; # 0.2 uA / mm^2 current for 10 ms
V, m, n, h = hh(I, dt) # simulate it!


In [ ]:
# Generate a two y-axis plot with current (red) and resulting membrane potential (blue)
fig, ax1 = plt.subplots()
t = np.arange(0, 1, dt / 1000)
ax1.plot(t, V, 'b-')
ax1.set_xlabel('time (s)')
ax1.set_ylabel(r'$V_{m}$', color='b')
ax1.set_ylim([-100, 80])
ax2 = ax1.twinx()
ax2.plot(t, I, 'r-')
ax2.set_ylim([-0.5, 5])
ax2.set_ylabel(r'$I_{in}$', color='r')
ax1.set_xlim([0.1, 0.3])
